In [2]:
import pandas as pd
import os
import sys
import time
%matplotlib inline
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from client import * #client credentials here

In [3]:
# We create the client object to interact with the API using Spotipy

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id =Client_id,client_secret=Client_secret ,redirect_uri=Redirect_uri ))

In [4]:
# We bring in the main DF where we have the chosen artists

df = pd.read_csv('graph_metrics.csv')
df.head()

,name,degree_centrality,num_of_edges,betweenness_centrality,closeness_centrality,eigenvector_centrality,page_rank,ID
0,Charles Ans,0.026627,9,0.006667,0.248529,0.020963,0.003390,1
1,Nanpa Básico,0.011834,4,0.000929,0.241601,0.004618,0.001662,2
2,Rapsusklei,0.079882,27,0.039389,0.261813,0.233407,0.008371,3
3,BCN,0.017751,6,0.000430,0.222368,0.019832,0.002334,4
4,Neto Peña,0.020710,7,0.002914,0.246356,0.007073,0.002725,5


In [6]:
df.drop(['eigenvector_centrality','ID','num_of_edges'], axis= 1, inplace = True)

In [7]:
df.shape

(339, 5)

In [9]:
# FUNCIONES PARAMETROS ARTISTAS

def funcion_popularidad(artist):

    #     te dice el URI del artista
    busqueda_id = sp.search(q=f'artist: {artist}',limit = 1, type='artist')
    try:
        ID = busqueda_id['artists']['items'][0]['uri']
    except IndexError:
        fake_artists.append(artist)
        return       
           
    #     Definimos la parametros del artista

    artist_param = sp.artist(ID)

    #    Definimos los diccionarios donde irán los parametros

    return artist_param['popularity']
    
def funcion_followers(artist):

    #     te dice el URI del artista
    busqueda_id = sp.search(q=f'artist: {artist}',limit = 1, type='artist')
    try:
        ID = busqueda_id['artists']['items'][0]['uri']
    except IndexError:
        fake_artists.append(artist)
        return       
           
    #     Definimos la parametros del artista

    artist_param = sp.artist(ID)

    #    Definimos los diccionarios donde irán los parametros

    return artist_param['followers']['total']

    
def funcion_generos(artist):

    #     te dice el URI del artista
    busqueda_id = sp.search(q=f'artist: {artist}',limit = 1, type='artist')
    try:
        ID = busqueda_id['artists']['items'][0]['uri']
    except IndexError:
        fake_artists.append(artist)
        return       
           
    #     Definimos la parametros del artista

    artist_param = sp.artist(ID)

    #    Definimos los diccionarios donde irán los parametros

    return artist_param['genres']


In [10]:
funcion_popularidad('Charles Ans')

74

In [11]:
# Create the popularity metric
df['popularity'] = [funcion_popularidad(i) for i in df['name']]

In [12]:
# Create the followers metric
df['followers'] = [funcion_followers(i) for i in df['name']]

In [13]:
# Create the genres metric
df['genres'] = [funcion_generos(i) for i in df['name']]

In [14]:
df.head()

,name,degree_centrality,betweenness_centrality,closeness_centrality,page_rank,popularity,followers,genres
0,Charles Ans,0.026627,0.006667,0.248529,0.003390,74,1104527,"[latin hip hop, mexican hip hop, perreo, rap c..."
1,Nanpa Básico,0.011834,0.000929,0.241601,0.001662,76,1585468,"[colombian hip hop, mexican hip hop]"
2,Rapsusklei,0.079882,0.039389,0.261813,0.008371,57,301652,"[boom bap espanol, rap conciencia, spanish hip..."
3,BCN,0.017751,0.000430,0.222368,0.002334,57,2933,[]
4,Neto Peña,0.020710,0.002914,0.246356,0.002725,77,821168,[mexican hip hop]


In [15]:
df.to_csv('df_all_metrics.csv', index = False)

In [16]:
# Checking the correlation

df.corr()

,degree_centrality,betweenness_centrality,closeness_centrality,page_rank,popularity,followers
degree_centrality,1.000000,0.647098,0.530211,0.938904,0.126504,0.043452
betweenness_centrality,0.647098,1.000000,0.580999,0.766424,0.193957,0.026923
closeness_centrality,0.530211,0.580999,1.000000,0.528807,0.293225,0.001591
page_rank,0.938904,0.766424,0.528807,1.000000,0.211591,0.060817
popularity,0.126504,0.193957,0.293225,0.211591,1.000000,0.446058
followers,0.043452,0.026923,0.001591,0.060817,0.446058,1.000000


In [17]:
top_10_popularity = (df.sort_values(by = ['popularity'], ascending = False)).head(10)
top_10_popularity

,name,degree_centrality,betweenness_centrality,closeness_centrality,page_rank,popularity,followers,genres
296,Bad Bunny,0.041420,0.022892,0.262219,0.005856,100,47013944,"[latin, reggaeton, trap latino]"
334,The Weeknd,0.002959,0.000000,0.194365,0.000862,97,43976282,"[canadian contemporary r&b, canadian pop, pop]"
239,Rauw Alejandro,0.029586,0.010686,0.265306,0.004149,93,10916410,"[puerto rican pop, trap latino]"
48,ROSALÍA,0.011834,0.011679,0.241084,0.002521,89,5289326,[r&b en espanol]
303,Anitta,0.005917,0.000381,0.242294,0.001031,88,11551584,"[funk carioca, funk pop, pagode baiano, pop na..."
301,Justin Quiles,0.041420,0.016795,0.266142,0.005737,88,3434204,"[latin, reggaeton, trap latino]"
191,Sebastian Yatra,0.023669,0.008906,0.259401,0.003373,87,18807867,"[colombian pop, latin, reggaeton colombiano]"
187,Maria Becerra,0.020710,0.011455,0.267829,0.003174,87,2368358,[pop argentino]
250,Duki,0.035503,0.016659,0.283557,0.004948,87,5747196,"[argentine hip hop, trap argentino, trap latin..."
182,Nicky Jam,0.020710,0.005915,0.256449,0.002876,87,16724340,"[latin, latin hip hop, reggaeton, trap latino]"


In [18]:
bottom_10_popularity = (df.sort_values(by = ['popularity'], ascending = True)).head(10)
bottom_10_popularity

,name,degree_centrality,betweenness_centrality,closeness_centrality,page_rank,popularity,followers,genres
332,Red Light Studio,0.002959,0.000000,0.186021,0.000783,25,13,[]
190,JA Beatz,0.002959,0.000000,0.135308,0.001380,25,59,[]
227,Xmaga,0.008876,0.000000,0.151028,0.001671,27,37,[]
220,Heiss,0.014793,0.000769,0.187569,0.002762,28,41,[]
321,Swan Fyahbwoy,0.002959,0.000000,0.243867,0.000649,28,42551,[]
255,Vera GRV,0.005917,0.000000,0.230402,0.001113,29,634,[]
145,Noult,0.050296,0.006473,0.242642,0.004479,30,674,[]
148,Antonio Torres,0.047337,0.000080,0.222076,0.004133,30,345,[]
135,Roger Martinez,0.047337,0.000080,0.222076,0.004133,30,56,[]
275,Staid Beats,0.002959,0.000000,0.197892,0.000651,30,23,[]
